In [74]:
import spotipy
import spotipy.util as util
import pickle
import sys
import requests
from tqdm import tqdm
import os
import pandas as pd
import numpy as np
from multiprocessing import Pool
from fuzzywuzzy import fuzz
import string
import json
import unidecode

In [183]:


# Spotify Infos
scope = 'user-library-read'
username = "ilias.miraoui@gmail.com"

token = util.prompt_for_user_token(username, scope, client_id='6d6b00a4a500471e94c5a3f1559e8cc6', client_secret='500950b79db94ac88f6859d24e4e6033', redirect_uri='http://localhost:8888/callback')
if token:
    spotify = spotipy.Spotify(auth=token)
else:
    print("Connexion to spotify API failed.")
    sys.exit()

    


In [76]:
def simplify_string(s):
    return unidecode.unidecode(s.lower()).replace('the', ' ').replace(' ', '').translate(str.maketrans('', '', string.punctuation))


def similar(a, b, threshold=80):
    return fuzz.ratio(simplify_string(a), simplify_string(b)) >= threshold

In [105]:
def get_song_details(artist,track):
    song_details = dict()
    results = spotify.search(q=" track:" + track + ' artist:' + artist  , type='track')
    maxi = None
    if results['tracks']['total'] > 0:
        for i in results['tracks']['items']:
            if similar(track, i['name'], 80) and similar(artist, i["artists"][0]["name"], 80):
                maxi = i
                break
        if maxi is None:
            return None
        if results is None or len(results) == 0:
            return None
        _id = maxi["id"] 
        song_details[_id] = dict()
        song_details[_id]["explicit"] = maxi["explicit"]
        song_details[_id]["duration_ms"] = maxi["duration_ms"]
        song_details[_id]["disc_number"] = maxi["disc_number"]
        song_details[_id]["track_number"] = maxi["track_number"]
    return song_details


    

In [201]:
def get_song_features(_id):
    song_features = dict()
    results = spotify.audio_features(_id)
    results = results[0]
    if results != None: 
        song_features["danceability"] = results["danceability"]
        song_features["energy"] = results["energy"]
        song_features["key"] = results["key"]
        song_features["loudness"] = results["loudness"]
        song_features["mode"] = results["mode"]
        song_features["speechiness"] = results["speechiness"]
        song_features["acousticness"] = results["acousticness"]
        song_features["instrumentalness"] = results["instrumentalness"]
        song_features["liveness"] = results["liveness"]
        song_features["valence"] = results["valence"]
        song_features["tempo"] = results["tempo"]
        song_features["time_signature"] = results["time_signature"]
        return song_features
    else:
        return None

In [79]:
song = get_song_details("Jimi Hendrix","Purple Haze")
features = get_song_features(song.keys())

In [80]:
song.keys()

dict_keys(['0wJoRiX5K5BxlqZTolB2LD'])

In [83]:
with open('billboard-200_data.pkl', 'rb') as f:
        df = pickle.load(f)
with open('billboard-200_chart.pkl', 'rb') as f:
        chart = pickle.load(f)

In [85]:
song_df = df.drop_duplicates(["artist","title"])
song_df

In [177]:
with open('song_df.pkl', 'wb') as f:
    pickle.dump(song_df, f)

In [175]:
for song in song_df.index[17700:]:
    if pd.isna(song_df.loc[song,"spotify_id"]) == True:
        title = song_df.loc[song,"title"]
        artist = song_df.loc[song,"artist"]
        song_details = get_song_details(artist,title)
        if type(song_details) == dict and len(song_details)>0 :
            song_df.loc[song,"spotify_id"] = list(song_details.keys())[0]
            for feature in list(song_details[list(song_details.keys())[0]].keys()):
                song_df.loc[song,"spotify_"+feature] = song_details[list(song_details.keys())[0]][feature]

retrying ...1secs
retrying ...1secs


In [177]:
with open('song_df.pkl', 'wb') as f:
    pickle.dump(song_df, f)

In [177]:
with open('song_df.pkl', 'wb') as f:
    pickle.dump(song_df, f)

In [206]:
for song in song_df[song_df["spotify_id"].notnull()].index:
    spotify_key = song_df.loc[song,"spotify_key"]
    spotify_id = song_df.loc[song,"spotify_id"]
    if pd.isna(spotify_key) == True:
        song_features = get_song_features(spotify_id)
        if song_features != None:
            for feature in list(song_features.keys()):
                song_df.loc[song,"spotify_"+feature] = song_features[feature]

In [205]:
song_df.dropna()

,artist,title,isnew,rank,date,spotify_id,spotify_explicit,spotify_duration_ms,spotify_disc_number,spotify_track_number,...,spotify_key,spotify_loudness,spotify_mode,spotify_speechiness,spotify_acousticness,spotify_instrumentalness,spotify_liveness,spotify_valence,spotify_tempo,spotify_time_signature
4,Summer Walker,Over It,False,5,2019-11-16,0csmcZqM3uTKfjxehw0Lw2,True,131106.0,1.0,1.0,...,4.0,-8.113,0.0,0.1250,0.23100,0.000000,0.0885,0.4550,131.996,4.0
7,Taylor Swift,Lover,False,8,2019-11-16,1dGr1c8CrMLDpV6mPbImSI,False,221306.0,1.0,3.0,...,7.0,-7.582,1.0,0.0919,0.49200,0.000016,0.1180,0.4530,68.534,4.0
9,Chris Brown,Indigo,False,10,2019-11-16,6ZOXF7nbzCpPVNlC0AG1rF,True,192626.0,1.0,1.0,...,3.0,-4.791,0.0,0.1220,0.32800,0.000000,0.2470,0.5260,151.944,4.0
15,Lizzo,Cuz I Love You,False,16,2019-11-16,6YdQgWSpsxhVeX6Xmv3IFJ,True,179978.0,1.0,1.0,...,1.0,-4.039,1.0,0.0479,0.01560,0.000000,0.2340,0.5200,162.159,3.0
17,Luke Combs,This One's For You,False,18,2019-11-16,02Ey48n33H3erWJRZAtTFj,False,230240.0,1.0,9.0,...,11.0,-7.811,1.0,0.0449,0.44400,0.000001,0.1080,0.2350,149.278,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127990,Hilary Duff,Dignity,False,191,2007-08-25,6PeNKjSme3ZlGD5Lf8KFYg,False,194933.0,1.0,2.0,...,2.0,-2.822,0.0,0.0355,0.00453,0.000022,0.2580,0.6330,127.008,4.0
127994,Within Temptation,The Heart Of Everything,False,195,2007-08-25,2JLdwAyDK1rkd4KAlw39XT,False,334813.0,1.0,5.0,...,8.0,-5.515,1.0,0.1840,0.01410,0.005040,0.1390,0.1460,166.014,4.0
127995,Red Hot Chili Peppers,Stadium Arcadium,False,196,2007-08-25,4y84ILALZSa4LyP6H7NVjR,False,314773.0,1.0,4.0,...,9.0,-3.925,1.0,0.0318,0.02360,0.000003,0.1560,0.2190,113.474,4.0
128083,Kia Shine,Due Season,True,84,2007-08-18,1IhuQ26JpHoK0eSSXJwp21,True,693373.0,1.0,15.0,...,9.0,-3.652,0.0,0.2420,0.12000,0.000000,0.1650,0.4740,152.534,4.0
